In [52]:
import json
import torch
import matplotlib.pyplot as plt

In [53]:
with open("../datasets/bdd100k/labels/bdd100k_labels_images_train.json") as f:
    data = json.load(f)

In [54]:
len(data)

69863

In [55]:
def get_all_filenames(dir):
    import os

    filenames = []
    for root, dirs, files in os.walk(dir):
        for file in files:
            filenames.append(file)
    return filenames

In [62]:
images = set(get_all_filenames("../datasets/bdd100k/images/10k/train"))
len(images)

7000

In [65]:
train_images = set([d["name"] for d in data])

In [66]:
for i in images:
    if i not in train_images:
        print(i)

29459bfc-00000000.jpg
60e9c379-00000000.jpg
07e7f300-544c108f.jpg
3be73583-32d32f05.jpg
1ee2d0f9-c1f48031.jpg
54ae1163-00000000.jpg
6d42bf70-34acb0fa.jpg
638f30e3-22474f13.jpg
56a80277-d8c0d525.jpg
54f36f5c-71638bc5.jpg
76a0a08f-00000000.jpg
5c59c5f8-31d384cd.jpg
57e171f1-00d4df18.jpg
51a9ece0-0e4b8c3a.jpg
c0a7a9dc-60feea97.jpg
6fdfef8c-818c9876.jpg
23fe59f4-66b4f566.jpg
428987f1-00000000.jpg
bd6c7aef-f3373443.jpg
4ec8631a-c5f7326a.jpg
00e9be89-00001650.jpg
2f26b25d-3de7fde6.jpg
3a0c2c74-ec02b465.jpg
2e7128c1-15fd7de8.jpg
307be9b0-6e850001.jpg
30b4085c-2a3ca635.jpg
059286c1-74192283.jpg
51d80c39-86946da2.jpg
75166788-59f22c0b.jpg
084811da-94a1a9ee.jpg
0c7aa2a6-9c0f3111.jpg
1294a00a-b613ccf0.jpg
0f1a2d28-63e10000.jpg
2c96a31b-35a70001.jpg
304c7e0b-cbd1ec1a.jpg
72d1a01b-4a230001.jpg
2df265da-e4a8fdf3.jpg
5363f06b-00000000.jpg
00e9be89-00001905.jpg
06e41b2b-a0f1935f.jpg
019891ed-c14e0001.jpg
05ff3678-eb8a12cd.jpg
2400952e-00000000.jpg
27c0e9a0-0898a6a9.jpg
1d9d905a-c2a16563.jpg
47ac6ca9-0

In [59]:
with open("../datasets/bdd100k/labels/bdd10k_labels_images_val.json", "w") as f:
    json.dump(data, f)

KeyboardInterrupt: 

In [ ]:
reduced_data = []

for image in data:
    reduced_data.append(
        {
            "name": image["name"],
            "attributes": {"timeofday": image["attributes"]["timeofday"]},
            "labels": [
                {"category": label["category"], "box2d": label["box2d"]}
                for label in image["labels"]
                if "box2d" in label and label["box2d"] is not None
            ],
        }
    )

with open(
    "../datasets/bdd100k/labels/bdd100k_labels_images_train_reduced.json", "w"
) as f:
    json.dump(reduced_data, f)

In [ ]:
def display_image(image_name):
    plt.imshow(plt.imread("../dataset/bdd100k/images/100k/train/" + image_name))
    plt.show()

In [ ]:
categorie_to_id = {
    "bike": 0,
    "bus": 1,
    "car": 2,
    "motor": 3,
    "person": 4,
    "rider": 5,
    "traffic light": 6,
    "traffic sign": 7,
    "train": 8,
    "truck": 9,
    "pedestrian": 10,
    "motorcycle": 11,
    "bicycle": 12,
}

In [ ]:
day_images = set()
night_images = set()

for image in data:
    if image["attributes"]["timeofday"] == "daytime":
        day_images.add(image["name"])
    elif image["attributes"]["timeofday"] == "night":
        night_images.add(image["name"])

In [ ]:
def to_tensor(images, score=False, weather=None, timeofday=None):
    sorted_images = sorted(images, key=lambda x: x["name"])

    results = []
    for img in sorted_images:
        if score:
            if timeofday == "daytime" and img["name"] not in day_images:
                continue
            if timeofday == "night" and img["name"] not in night_images:
                continue
        else:
            if weather is not None and img["attributes"]["weather"] != weather:
                continue
            if timeofday is not None and img["attributes"]["timeofday"] != timeofday:
                continue

        labels = img["categories"]
        labels = [l for l in labels if "box2d" in l and l["box2d"] is not None]
        boxes = [l["box2d"] for l in labels]
        boxes = [[b["x1"], b["y1"], b["x2"], b["y2"]] for b in boxes]
        boxes = torch.tensor(boxes)

        categories = [l["category"] for l in labels]
        categories = [categorie_to_id[c] for c in categories]
        categories = torch.tensor(categories)

        result = {"boxes": boxes, "labels": categories}

        if score:
            scores = [l["score"] for l in labels]
            scores = torch.tensor(scores)
            result["scores"] = scores

        results.append(result)

    return results

In [ ]:
gt = to_tensor(data)
day = to_tensor(data, timeofday="daytime")
night = to_tensor(data, timeofday="night")
len(day), len(night)

(2526, 115)

In [ ]:
from torchmetrics.detection import MeanAveragePrecision

In [ ]:
def compute_map(baseline):
    with open(f"../baselines/{baseline}.json") as f:
        baseline = json.load(f)

    day_preds = to_tensor(baseline["frames"], score=True, timeofday="daytime")
    night_preds = to_tensor(baseline["frames"], score=True, timeofday="night")

    iou_thresholds = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
    day_map = MeanAveragePrecision("xyxy", "bbox", iou_thresholds=iou_thresholds)
    night_map = MeanAveragePrecision("xyxy", "bbox", iou_thresholds=iou_thresholds)

    print(len(day_preds), len(night_preds))

    day_map.update(day_preds, day)
    night_map.update(night_preds, night)

    return day_map.compute(), night_map.compute()

In [ ]:
result = compute_map("cascade_rcnn_swin-t_fpn_3x_det_bdd100k")
print(result)

0 0


ValueError: Expected argument `preds` and `target` to have the same length, but got 0 and 2526

In [ ]:
print(
    "Daytime mAP: {} mAP_50: {}, mAP_75: {}".format(
        result[0]["map"], result[0]["map_50"], result[0]["map_75"]
    )
)
print(
    "Nighttime mAP: {} mAP_50: {}, mAP_75: {}".format(
        result[1]["map"], result[1]["map_50"], result[1]["map_75"]
    )
)

Daytime mAP: 0.2511175274848938 mAP_50: 0.41557979583740234, mAP_75: 0.25299593806266785
Nighttime mAP: 0.20862507820129395 mAP_50: 0.3685239851474762, mAP_75: 0.19639737904071808


In [ ]:
# ConvNeXt + R-CNN
# Daytime mAP: 0.26108455657958984 mAP_50: 0.42570289969444275, mAP_75: 0.26357346773147583
# Nighttime mAP: 0.20968283712863922 mAP_50: 0.36812451481819153, mAP_75: 0.19964323937892914

# RetinaNet
# Daytime mAP: 0.23603859543800354 mAP_50: 0.405560702085495, mAP_75: 0.23004977405071259
# Nighttime mAP: 0.19778987765312195 mAP_50: 0.3620649576187134, mAP_75: 0.1865830421447754